In [ ]:
from collections import namedtuple

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

In [ ]:
from sympy.physics.units import newton, meter
mm = meter / 1000
MPa = 1e6 * newton / meter**2
GPa = 1e9 * newton / meter**2

In [ ]:
def load(file):
    data = pd.read_csv(file, index_col=0, skiprows=94)

#     data['Extension mm'] *= mm
#     data['Load kgf'] *= newton
    data['stress'] = data['Compressive stress MPa']
    data['strain'] = data['Compressive strain %']
    return data

In [ ]:
sample1 = load('data/Packing_Peanuts_Instron/Starch_1_raw.is_comp.raw')

In [ ]:
pd.Insample1['Extension mm'].get_loc('Test Type')

In [ ]:
sample1

In [ ]:
def process(data, area, length):
    data['stress'] = data.load / area
    data['strain'] = data.extension / length

process(sample1, 10*mm**2, 10*mm)

In [ ]:
samples = []

for file, area, length in [
    ('data/Packing_Peanuts_Instron/Starch_1_raw.is_comp_RawData/Specimen_RawData_1.csv', 10*mm**2, 10*mm),
    ('data/Packing_Peanuts_Instron/Starch_1.is_comp_RawData/Specimen_RawData_2.csv', 10*mm**2, 10*mm),
    ('data/Packing_Peanuts_Instron/Starch_1.is_comp_RawData/Specimen_RawData_3.csv', 10*mm**2, 10*mm),
    ('data/Packing_Peanuts_Instron/Starch_1.is_comp_RawData/Specimen_RawData_4.csv', 10*mm**2, 10*mm),
]:
    sample = load(file)
    process(sample, area, length)
    samples.append(sample)

In [ ]:
samples[0].head()

In [ ]:
fig = plt.figure(num=None, figsize=(10, 6), dpi=1000, facecolor='w', edgecolor='k')

plt.xlabel('Strain (mm)')
plt.ylabel('Stress (MPa)')
plt.title('Biodegradable packing peanuts')

for i, sample in enumerate(samples):
    plt.plot(-sample.strain, -sample.stress / MPa, label=f'Trial {i+1}')

plt.legend();